In [1]:
import numpy as np

In [ ]:

def filteredUpdate(target, current, filter):
    """
    Calculates a filtered percental update
    target: T
    current: T
    filter: double
    """
    return (1 - filter) * current + filter * target

def filterStep(update_frequency, filter_percentage):
    """
    Calculates the filter step
    update_frequency: double
    filter_percentage: double
    """
    kappa = -1/ (np.log(1 - min(filter_percentage, 0.999999)))
    return 1/ (kappa * update_frequency + 1.0)

def saturateValue(x, x_min, x_max):
    """
    Saturate a variable x with the limits x_min and x_max
    x: double
    x_min: double
    x_max: double
    """
    return min(max(x, x_min), x_max)

def saturateTorqueRate(tau_d_calculated, tau_d_saturated, delta_tau_max):
    """
    Saturate torque rate to not stress the motors
    tau_d_calculated: vector
    tau_d_saturated: vector 
    delta-tau_max: double 
    """
    for i in range(tau_d_calculated.shape[0]):
        difference = tau_d_calculated[i] - tau_d_saturated[i]
        tau_d_saturated[i] += saturateValue(difference -delta_tau_max, delta_tau_max)

class CartesianImpedanceController:
    def __init__(self):
        pass

    def initDesiredPose(self, position_d_target):
        self._cartesian_stiffness_target = 1 

    def setNumberOfJoints(self, n_joints):
        """
        n_joints: int
        """
        if n_joints < 0:
            raise Exception("Number of joints must be positive")

        self._n_joints = n_joints
        self._q = np.zeros((n_joints,))
        self._dq = np.zeros((n_joints,))
        self._jacobian = np.zeros((6, n_joints))
        self._tau_c = np.zeros((n_joints,))

    def setStiffnessInternal(self, stiffness, auto_damping):
        """
        stiffness: 3x1 vector
        auto_damping: boolean
        """
        if (stiffness < 0).any():
            raise Exception("Stiffness values must be positive")
        np.fill_diagonal(stiffness)
        
        if auto_damping:
            self.applyDamping()

    def setStiffness_big(self, t_x, t_y, t_z, auto_dam):
        """
        Idk what to call this

        t_x: double
        t_y: double
        t_z: double
        """
        stiffness = np.array([t_x, t_y, t_z]) 
        self.setStiffnessInternal(stiffness, auto_dam)

    def setDampingFactors(self, d_x, d_y, d_z):
        """
        d_x: double
        d_y: double
        d_z: double
        """
        damping = np.array([d_x, d_y, d_z])
        self.setDampingFactorsInternal(damping)
        if (damping < 0).any():
            raise Exception("Dampin gfactor must not be negative")
        self._damping_factors = damping
        self.applyDamping()

    def applyDamping(self):
        cartesian_stiffness_target_diag = np.diag(self._cartesian_stiffness_target)
        np.fill_diagonal(self._damping_factors * self.dampingRule(cartesian_stiffness_target_diag))
    
    def setReferencePose(self, position_d_target):
        self.Pposition_d_target = position_d_target

    def setFiltering(self, update_frequency, filter_params_stiffness, filter_params_pose, filter_params_wrench):
        self.setUpdateFrequency(update_frequency) 
    